In [1]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Flatten, Dense, Input, Dropout, Rescaling
from tensorflow.keras.models import Model

In [2]:
notebook_path = os.path.dirname(os.path.abspath('brain_MRI_classification.ipynb'))
datasets_combined = os.path.join(notebook_path, 'brainMRI_data')

train_directory = os.path.join(datasets_combined, 'Training')
test_directory = os.path.join(datasets_combined, 'Testing')

In [3]:
BATCH_SIZE = 64
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(train_directory,
                                             batch_size = BATCH_SIZE,
                                             image_size = IMG_SIZE,
                                             shuffle = True,
                                             validation_split = 0.2,
                                             subset = 'training',
                                             seed = 42,
                                             label_mode='categorical')

validation_dataset = image_dataset_from_directory(train_directory,
                                                  batch_size = BATCH_SIZE,
                                                  image_size = IMG_SIZE,
                                                  shuffle = True,
                                                  validation_split = 0.2,
                                                  subset = 'validation',
                                                  seed = 42,
                                                  label_mode='categorical')

Found 2870 files belonging to 4 classes.
Using 2296 files for training.
Found 2870 files belonging to 4 classes.
Using 574 files for validation.


In [4]:
test_dataset = image_dataset_from_directory(test_directory,
                                            shuffle = False,
                                            image_size = IMG_SIZE,
                                            label_mode='categorical')

Found 394 files belonging to 4 classes.


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [6]:
# (224, 224) -> (224, 224, 3) for the 3 color channels
IMG_SHAPE = IMG_SIZE + (3,)

# load network without the top classification layers
base_model = VGG19(include_top = False,
                   weights = 'imagenet',
                   input_shape=IMG_SHAPE)

In [7]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.1),
])

In [8]:
# only augment training set
train_augmented = train_dataset.map(lambda x, y: (data_augmentation(x, training = False), y))

In [9]:
preprocess_input = tf.keras.applications.vgg19.preprocess_input

In [12]:
base_model.trainable = False

inputs = tf.keras.Input(shape=IMG_SHAPE)
x = preprocess_input(inputs)
x = base_model(x, training=False)

# Flatten the output layer to 1D
x = Flatten()(x)

# Add a fully connected layer with 4096 hidden units, ReLU activation
x = Dense(4096, activation = 'relu')(x)

# Add a dropout layer with 0.2 (20%) rate
x = Dropout(0.2)(x) 

# Add another FC layer, 4096 units, ReLU activation
x = Dense(4096, activation = 'relu')(x)

# Add another dropout layer with 0.2 (20%) rate
x = Dropout(0.2)(x) 

# Add a final FC layer for classification with 4 units using softmax activation function
outputs = Dense(4, activation = 'softmax')(x)

In [15]:
# Configure and compile the model
model_new = Model(inputs, outputs)
model_new.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),
               metrics=['accuracy'])

In [16]:
results_new = model_new.fit(train_augmented, epochs = 20, validation_data = validation_dataset, verbose = 1)

Epoch 1/20
36/36 [==============================] - 626s 18s/step - loss: 6.4833 - accuracy: 0.6677 - val_loss: 1.0843 - val_accuracy: 0.8171
Epoch 2/20
36/36 [==============================] - 770s 21s/step - loss: 0.4259 - accuracy: 0.9225 - val_loss: 0.3853 - val_accuracy: 0.9111
Epoch 3/20
36/36 [==============================] - 785s 22s/step - loss: 0.0784 - accuracy: 0.9734 - val_loss: 0.3642 - val_accuracy: 0.9129
Epoch 4/20
36/36 [==============================] - 791s 22s/step - loss: 0.0501 - accuracy: 0.9865 - val_loss: 0.4678 - val_accuracy: 0.9164
Epoch 5/20
36/36 [==============================] - 793s 22s/step - loss: 0.0475 - accuracy: 0.9887 - val_loss: 0.4543 - val_accuracy: 0.9286
Epoch 6/20
36/36 [==============================] - 797s 22s/step - loss: 0.0648 - accuracy: 0.9882 - val_loss: 0.3794 - val_accuracy: 0.9338
Epoch 7/20
36/36 [==============================] - 798s 22s/step - loss: 0.0213 - accuracy: 0.9939 - val_loss: 0.3945 - val_accuracy: 0.9024
Epoch 

In [17]:
base_model.trainable = False

inputs = tf.keras.Input(shape=IMG_SHAPE)
x = preprocess_input(inputs)
x = base_model(x, training=False)

# Flatten the output layer to 1D
x = Flatten()(x)

# Add a fully connected layer with 4096 hidden units, ReLU activation
x = Dense(4096, activation = 'relu')(x)

# Add a dropout layer with 0.2 (20%) rate
x = Dropout(0.5)(x) 

# Add another FC layer, 4096 units, ReLU activation
x = Dense(4096, activation = 'relu')(x)

# Add another dropout layer with 0.2 (20%) rate
x = Dropout(0.5)(x) 

# Add a final FC layer for classification with 4 units using softmax activation function
outputs = Dense(4, activation = 'softmax')(x)

# Configure and compile the model
model_new2 = Model(inputs, outputs)
model_new2.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),
               metrics=['accuracy'])

In [20]:
results_new2 = model_new2.fit(train_augmented, epochs = 10, validation_data = validation_dataset, verbose = 1)

Epoch 1/10
36/36 [==============================] - 366s 10s/step - loss: 2.5798 - accuracy: 0.8005 - val_loss: 1.1642 - val_accuracy: 0.8659
Epoch 2/10
36/36 [==============================] - 389s 11s/step - loss: 1.3467 - accuracy: 0.8754 - val_loss: 1.1656 - val_accuracy: 0.8780
Epoch 3/10
36/36 [==============================] - 426s 12s/step - loss: 1.2034 - accuracy: 0.8981 - val_loss: 2.1256 - val_accuracy: 0.8728
Epoch 4/10
36/36 [==============================] - 449s 13s/step - loss: 0.7628 - accuracy: 0.9216 - val_loss: 0.8045 - val_accuracy: 0.9111
Epoch 5/10
36/36 [==============================] - 459s 13s/step - loss: 0.6814 - accuracy: 0.9368 - val_loss: 2.4354 - val_accuracy: 0.8118
Epoch 6/10
36/36 [==============================] - 471s 13s/step - loss: 0.5718 - accuracy: 0.9395 - val_loss: 2.2370 - val_accuracy: 0.8711
Epoch 7/10
36/36 [==============================] - 460s 13s/step - loss: 0.4314 - accuracy: 0.9578 - val_loss: 0.6484 - val_accuracy: 0.9216
Epoch 